# Importacion de librerias

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
random.seed(42)

pd.options.display.float_format = '{:.5f}'.format



# Obtencion de datos

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:
tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Indices_group' in symbol.path]
tickers

['AUS200m',
 'DE30m',
 'FR40m',
 'HK50m',
 'JP225m',
 'STOXX50m',
 'UK100m',
 'US30m',
 'US500m',
 'USTECm',
 'IN50m']

In [5]:

tickers = ['AUS200m','DE30m','FR40m','HK50m','JP225m','STOXX50m','UK100m','US30m','US500m','USTECm']

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2014, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

timeframes = [
    mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H4,
]

symbols = get_data(tickers, timeframes, date_from, date_to)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AUS200m
AUS200m
AUS200m
AUS200m
AUS200m
DE30m
DE30m
DE30m
DE30m
DE30m
FR40m
FR40m
FR40m
FR40m
FR40m
HK50m
HK50m
HK50m
HK50m
HK50m
JP225m
JP225m
JP225m
JP225m
JP225m
STOXX50m
STOXX50m
STOXX50m
STOXX50m
STOXX50m
UK100m
UK100m
UK100m
UK100m
UK100m
US30m
US30m
US30m
US30m
US30m
US500m
US500m
US500m
US500m
US500m
USTECm
USTECm
USTECm
USTECm
USTECm


In [6]:
max_start_date = None
intervals_start_dates = {}

for interval in timeframes:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            print(ticker)
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates

AUS200m


{15: Timestamp('2021-10-27 17:15:00'),
 30: Timestamp('2021-10-27 17:15:00'),
 16385: Timestamp('2021-10-27 17:15:00'),
 16386: Timestamp('2021-10-27 17:15:00'),
 16388: Timestamp('2021-10-27 17:15:00')}

In [7]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00')
limit_date_train

Timestamp('2022-10-01 00:00:00')

# Test preliminar

In [8]:
from backbone.utils.general_purpose import calculate_units_size, diff_pips


class BbandsCross(Strategy):
    risk = 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200

    atr_multiplier = 1.5
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )

        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)


    def next(self):
        actual_close = self.data.Close[-1]
        
        if self.position:
            if self.position.is_long:
                if crossover(self.data.Close, self.middle_band) or actual_close <= self.sma[-1]:
                    self.position.close()

            if self.position.is_short:
                if crossover(self.middle_band, self.data.Close) or actual_close >= self.sma[-1]:
                    self.position.close()

        else:

            if crossover(self.data.Close, self.lower_band) and actual_close > self.sma[-1]:
                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.buy(
                    size=units,
                    sl=sl_price
                )
                
            if crossover(self.upper_band, self.data.Close) and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.sell(
                    size=units,
                    sl=sl_price
                )

In [9]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    BbandsCross, 
]

experiments = parameter_combinations = list(itertools.product(
    tickers, timeframes, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    start_date = intervals_start_dates[interval]

    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        strategy,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

performance.drawdown_duration = pd.to_timedelta(performance.drawdown_duration)

performance.drawdown_duration = performance.drawdown_duration.dt.days



In [10]:
performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BbandsCross,AUS200m,15,0.73968,-23.30867,76691.33046,43.71037,307,45.53991,0.00000,213,-0.01845,7.90176,76691.33046,338 days 02:30:00,-0.53325,-2.79742
0,BbandsCross,AUS200m,30,0.82249,-23.00983,76990.16820,29.30873,325,40.74074,0.00000,108,-0.06376,8.04490,76990.16820,338 days 02:00:00,-0.78508,-3.56159
0,BbandsCross,AUS200m,16385,0.82111,-14.40448,85595.52143,19.55956,315,40.29851,0.00000,67,-0.11818,8.12813,85595.52143,337 days 10:00:00,-0.73644,-2.95628
0,BbandsCross,AUS200m,16386,0.00850,-0.04391,99956.08514,7.76428,311,43.75000,0.00000,32,-0.01877,7.91476,99956.08514,337 days 10:00:00,-0.00566,-0.01752
0,BbandsCross,AUS200m,16388,0.87058,7.00188,107001.87780,2.50022,89,76.92308,1.50865,13,0.39801,8.16761,107001.87780,337 days 08:00:00,2.80050,5.27920
0,BbandsCross,DE30m,15,0.91619,-47.46337,52536.63468,48.75749,331,38.36735,0.00000,245,-0.06857,9.32764,52536.63468,338 days 02:30:00,-0.97346,-5.25150
0,BbandsCross,DE30m,30,0.92201,-27.64171,72358.28749,32.00393,324,39.09774,0.00000,133,-0.07570,10.73319,72358.28749,338 days 02:00:00,-0.86370,-4.10208
0,BbandsCross,DE30m,16385,0.37381,-3.66373,96336.27358,7.48464,245,45.09804,0.00000,51,-0.03592,7.01149,96336.27358,337 days 10:00:00,-0.48950,-1.70617
0,BbandsCross,DE30m,16386,0.04818,-0.79243,99207.57360,5.81362,260,47.61905,0.00000,21,0.02796,6.81987,99207.57360,337 days 10:00:00,-0.13631,-0.35949
0,BbandsCross,DE30m,16388,0.01743,1.40181,101401.80510,2.82291,196,41.66667,0.34978,12,0.09543,5.20833,101401.80510,337 days 08:00:00,0.49658,0.94053


In [11]:

filter_performance = performance[
    (performance['return']>0) 
].sort_values(by=['drawdown_duration'], ascending=[True]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
timeframes = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BbandsCross,AUS200m,16388,0.87058,7.00188,107001.87780,2.50022,89,76.92308,1.50865,13,0.39801,8.16761,107001.87780,337 days 08:00:00,2.80050,5.27920
0,BbandsCross,STOXX50m,16388,0.00045,0.11947,100119.47200,2.82181,103,37.50000,0.02412,16,0.07170,9.51389,100119.47200,337 days 08:00:00,0.04234,0.08857
0,BbandsCross,USTECm,16388,0.04168,4.23140,104231.40407,2.64320,112,44.44444,0.67053,9,0.93630,9.35994,104231.40407,337 days 08:00:00,1.60087,2.67435
0,BbandsCross,HK50m,16388,0.50269,4.05957,104059.56879,4.00802,160,50.00000,0.83866,16,0.33991,7.26464,104059.56879,337 days 08:00:00,1.01286,2.29664
0,BbandsCross,US500m,16388,0.00116,5.88822,105888.22395,5.02962,166,43.75000,0.84418,16,0.56852,13.14522,105888.22395,337 days 08:00:00,1.17071,2.76677
0,BbandsCross,DE30m,16388,0.01743,1.40181,101401.80510,2.82291,196,41.66667,0.34978,12,0.09543,5.20833,101401.80510,337 days 08:00:00,0.49658,0.94053
0,BbandsCross,JP225m,16386,0.17464,3.75754,103757.54267,5.80182,202,52.38095,0.65126,21,0.08981,6.29051,103757.54267,337 days 10:00:00,0.64765,1.70759
0,BbandsCross,FR40m,16388,0.01678,1.12191,101121.90906,4.69145,218,37.50000,0.23603,16,0.08564,8.99654,101121.90906,337 days 08:00:00,0.23914,0.55849
0,BbandsCross,US30m,16388,0.01467,4.49055,104490.55302,6.22623,254,40.00000,0.66724,15,0.45065,13.35169,104490.55302,337 days 08:00:00,0.72123,1.72295


In [15]:
for ticker, interval in zip(portfolio, timeframes):
    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        BbandsCross,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )

    stats = bt_train.run()
    bt_train.plot(filename=f'./{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarn

# WFO

In [18]:
from backtest.utils import plot_full_equity_curve, walk_forward

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    

stats_per_symbol = {}
lookback_bars = 1000
validation_bars = 250
warmup_bars = 200

wfo_performance = pd.DataFrame()
for ticker, interval in zip(portfolio, timeframes):
    
    # print(ticker, interval)

    params = {
        'bband_std' : list(np.arange(1, 3, 0.2)),
        'maximize': optim_func_2
    }
    
    fracc_df = symbols[ticker][interval] * 0.01

    wfo_stats = walk_forward(
        BbandsCross,
        fracc_df, 
        lookback_bars=lookback_bars,
        validation_bars=validation_bars,
        warmup_bars=warmup_bars, 
        params=params,
        commission=7e-4, 
        margin=1/30, 
        cash=10_000,
        verbose=False
    )
    
    stats_per_symbol[ticker] = {}
    stats_per_symbol[ticker][interval] = wfo_stats
    
    df_equity = wfo_stats['_equity']
    
    plot_full_equity_curve(df_equity, title=f'{ticker}, {interval}')
    
    # Calculo el stability ratio
    x = np.arange(df_equity.shape[0]).reshape(-1, 1)
    reg = LinearRegression().fit(x, df_equity.Equity)
    stability_ratio = reg.score(x, df_equity.Equity)

    # Extraigo metricas
    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[wfo_stats['Return [%]']],
        'drawdown':[wfo_stats['Max. Drawdown [%]']],
        'final_eq':[wfo_stats['Equity Final [$]']],
        'drawdown_duration':[wfo_stats['Max. Drawdown Duration']],
        'win_rate':[wfo_stats['Win Rate [%]']], 
        'sharpe_ratio':[wfo_stats['Sharpe Ratio']],
        'trades':[wfo_stats['# Trades']],
        'avg_trade_percent':[wfo_stats['Avg. Trade [%]']],
        'exposure':[wfo_stats['Exposure Time [%]']],
        'final_equity':[wfo_stats['Equity Final [$]']],
        'Duration':[wfo_stats['Duration']],

    })

    wfo_performance = pd.concat([wfo_performance, df_stats])

wfo_performance['return/dd'] = wfo_performance['return'] / -wfo_performance['drawdown']
wfo_performance['drawdown'] = -wfo_performance['drawdown']
wfo_performance['custom_metric'] = (wfo_performance['return'] / (1 + wfo_performance.drawdown)) * np.log(1 + wfo_performance.trades)

wfo_performance.drawdown_duration = pd.to_timedelta(wfo_performance.drawdown_duration)
wfo_performance.drawdown_duration = wfo_performance.drawdown_duration.dt.days

wfo_performance.sort_values(by='return/dd', ascending=False)

{'bband_std': 1.5999999999999999}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.5999999999999999}
{'bband_std': 2.8}
{'bband_std': 2.8}
{'bband_std': 2.1999999999999997}
{'bband_std': 2.8}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.5999999999999999}
                         Equity  DrawdownPct DrawdownDuration
2022-09-06 20:00:00 10000.00000      0.00000              NaT
2022-09-07 00:00:00 10000.00000      0.00000              NaT
2022-09-07 04:00:00 10000.00000      0.00000              NaT
2022-09-07 08:00:00 10000.00000      0.00000              NaT
2022-09-07 12:00:00 10000.00000      0.00000              NaT
...                         ...          ...              ...
2024-08-19 16:00:00  9522.01254      0.00000              NaT
2024-08-19 20:00:00  9522.01254      0.00000              NaT
2024-08-20 00:00:00  9522.01254      0.00000              NaT
2024-08-20 04:00:

{'bband_std': 1.7999999999999998}
{'bband_std': 2.3999999999999995}
{'bband_std': 1.4}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.4}
{'bband_std': 1.4}
{'bband_std': 1.4}
{'bband_std': 1.4}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.2}
{'bband_std': 1.0}
{'bband_std': 2.3999999999999995}
                         Equity  DrawdownPct DrawdownDuration
2022-09-13 04:00:00 10000.00000      0.00000              NaT
2022-09-13 08:00:00 10000.00000      0.00000              NaT
2022-09-13 12:00:00 10000.00000      0.00000              NaT
2022-09-13 16:00:00 10049.39383      0.00000              NaT
2022-09-13 20:00:00 10030.69123      0.00186              NaT
...                         ...          ...              ...
2024-08-19 16:00:00  9445.40937      0.00868              NaT
2024-08-19 20:00:00  9445.40937      0.00868              NaT
2024-08-20 00:00:00  9445.40937      0.00868              NaT
2024-08-20 04:00:00  9445.40937      0.00868              NaT
2024-08-20 08:0

{'bband_std': 1.2}
{'bband_std': 1.0}
{'bband_std': 1.2}
{'bband_std': 1.2}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.7999999999999998}
{'bband_std': 2.5999999999999996}
{'bband_std': 2.5999999999999996}
{'bband_std': 2.5999999999999996}
{'bband_std': 2.8}
{'bband_std': 1.2}
{'bband_std': 1.2}
                         Equity  DrawdownPct DrawdownDuration
2022-09-21 04:00:00 10000.00000      0.00000              NaT
2022-09-21 08:00:00 10000.00000      0.00000              NaT
2022-09-21 12:00:00 10000.00000      0.00000              NaT
2022-09-21 16:00:00 10000.00000      0.00000              NaT
2022-09-21 20:00:00 10000.00000      0.00000              NaT
...                         ...          ...              ...
2024-08-19 16:00:00 10164.59261      0.00001              NaT
2024-08-19 20:00:00 10164.59261      0.00001              NaT
2024-08-20 00:00:00 10164.59261      0.00001              NaT
2024-08-20 04:00:00 10164.59261      0.00001              NaT
2024-08-20 08:0

{'bband_std': 1.4}
{'bband_std': 1.4}
{'bband_std': 1.4}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.9999999999999998}
{'bband_std': 2.5999999999999996}
{'bband_std': 2.5999999999999996}
{'bband_std': 1.5999999999999999}
{'bband_std': 2.8}
{'bband_std': 2.8}
                         Equity  DrawdownPct DrawdownDuration
2022-10-05 00:00:00 10000.00000      0.00000              NaT
2022-10-05 04:00:00 10000.00000      0.00000              NaT
2022-10-05 08:00:00 10000.00000      0.00000              NaT
2022-10-05 12:00:00 10000.00000      0.00000              NaT
2022-10-05 16:00:00 10000.00000      0.00000              NaT
...                         ...          ...              ...
2024-08-19 16:00:00  9690.27402      0.00000              NaT
2024-08-19 20:00:00  9690.27402      0.00000              NaT
2024-08-20 00:00:00  9690.27402      0.00000              NaT
2024-08-20 04:00:00  9690.27402      0.00000              NaT
2024-08-20 08:00:00

{'bband_std': 1.0}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.2}
{'bband_std': 2.8}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
                         Equity  DrawdownPct DrawdownDuration
2022-09-21 04:00:00 10000.00000      0.00000              NaT
2022-09-21 08:00:00 10000.00000      0.00000              NaT
2022-09-21 12:00:00 10000.00000      0.00000              NaT
2022-09-21 16:00:00 10000.00000      0.00000              NaT
2022-09-21 20:00:00 10000.00000      0.00000              NaT
...                         ...          ...              ...
2024-08-19 16:00:00 10811.74476      0.06137              NaT
2024-08-19 20:00:00 10811.74476      0.06137              NaT
2024-08-20 00:00:00 10811.74476      0.06137              NaT
2024-08-20 04:00:00 10811.74476      0.06137              NaT
2024-08-20 08:00:00 10811.74476      0.06137          15 days

[3000 rows x

{'bband_std': 1.9999999999999998}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.2}
{'bband_std': 1.2}
{'bband_std': 2.8}
{'bband_std': 2.8}
                         Equity  DrawdownPct DrawdownDuration
2022-09-13 08:00:00 10000.00000      0.00000              NaT
2022-09-13 12:00:00 10000.00000      0.00000              NaT
2022-09-13 16:00:00 10040.23468      0.00000              NaT
2022-09-13 20:00:00 10035.67468      0.00045              NaT
2022-09-14 00:00:00 10039.83568      0.00004              NaT
...                         ...          ...              ...
2024-08-19 16:00:00 10030.64599      0.00000              NaT
2024-08-19 20:00:00 10030.64599      0.00000              NaT
2024-08-20 00:00:00 10030.64599      0.00000              NaT
2024-08-20 04:00:00 10030.64599      0.00000              NaT


{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.2}
{'bband_std': 1.2}
{'bband_std': 1.2}
{'bband_std': 1.2}
{'bband_std': 2.3999999999999995}
{'bband_std': 2.3999999999999995}
{'bband_std': 2.5999999999999996}
{'bband_std': 2.5999999999999996}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.2}
{'bband_std': 1.2}
{'bband_std': 2.1999999999999997}
{'bband_std': 2.1999999999999997}
{'bband_std': 2.8}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.0}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.4}
                         Equity  DrawdownPct DrawdownDuration
2022-04-08 04:00:00 10000.00000      0.00000              NaT
2022-04-08 06:00:00 10000.00000      0.00000              NaT
2022-04-08 08:00:00 10000.00000      0.00000              NaT
2022-04-08 10:00:00 10000

{'bband_std': 2.3999999999999995}
{'bband_std': 1.4}
{'bband_std': 1.5999999999999999}
{'bband_std': 2.1999999999999997}
{'bband_std': 2.3999999999999995}
{'bband_std': 2.3999999999999995}
{'bband_std': 2.3999999999999995}
{'bband_std': 1.0}
{'bband_std': 1.2}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.2}
                         Equity  DrawdownPct DrawdownDuration
2022-09-09 12:00:00 10000.00000      0.00000              NaT
2022-09-09 16:00:00 10000.00000      0.00000              NaT
2022-09-11 20:00:00 10000.00000      0.00000              NaT
2022-09-12 00:00:00 10000.00000      0.00000              NaT
2022-09-12 04:00:00 10000.00000      0.00000              NaT
...                         ...          ...              ...
2024-08-19 16:00:00  9615.26887      0.01086              NaT
2024-08-19 20:00:00  9615.26887      0.01086              NaT
2024-08-20 00:00:00  9615.26887      0.01086              NaT
2024-08-20 04:00:00  9615.26887 

{'bband_std': 1.4}
{'bband_std': 1.5999999999999999}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.7999999999999998}
{'bband_std': 1.7999999999999998}
{'bband_std': 2.1999999999999997}
{'bband_std': 2.8}
{'bband_std': 2.8}
{'bband_std': 2.8}
{'bband_std': 2.8}
{'bband_std': 1.9999999999999998}
{'bband_std': 1.2}
                         Equity  DrawdownPct DrawdownDuration
2022-09-21 04:00:00 10000.00000      0.00000              NaT
2022-09-21 08:00:00 10000.00000      0.00000              NaT
2022-09-21 12:00:00 10000.00000      0.00000              NaT
2022-09-21 16:00:00 10000.00000      0.00000              NaT
2022-09-21 20:00:00 10000.00000      0.00000              NaT
...                         ...          ...              ...
2024-08-19 16:00:00  9933.13936      0.00000              NaT
2024-08-19 20:00:00  9933.13936      0.00000              NaT
2024-08-20 00:00:00  9933.13936      0.00000              NaT
2024-08-20 04:00:00  9933.13936      0.00000              NaT


,strategy,ticker,interval,stability_ratio,return,drawdown,final_eq,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BbandsCross,US500m,16388,0.40667,8.11745,6.13674,10811.74476,204,54.34783,0.57728,46,0.17383,4.63529,10811.74476,989 days 12:00:00,1.32276,4.37922
0,BbandsCross,USTECm,16388,0.02885,1.64593,7.49847,10164.59261,638,44.82759,0.16057,29,0.08289,3.36471,10164.59261,989 days 12:00:00,0.21950,0.65872
0,BbandsCross,DE30m,16388,0.14041,0.30646,6.48257,10030.64599,486,46.87500,0.00000,32,0.10103,3.69412,10030.64599,1001 days 08:00:00,0.04727,0.14320
0,BbandsCross,US30m,16388,0.23055,-0.66861,7.26118,9933.13936,491,33.33333,0.00000,24,0.04284,2.00000,9933.13936,989 days 12:00:00,-0.09208,-0.26052
0,BbandsCross,FR40m,16388,0.37019,-3.84731,9.36713,9615.26887,690,40.00000,0.00000,35,-0.05272,4.47059,9615.26887,1002 days 08:00:00,-0.41072,-1.32987
0,BbandsCross,AUS200m,16388,0.90615,-4.77987,9.55618,9522.01254,623,30.00000,0.00000,20,-0.10173,2.32941,9522.01254,1012 days 16:00:00,-0.50019,-1.37857
0,BbandsCross,HK50m,16388,0.44706,-3.09726,5.82447,9690.27402,574,37.50000,0.00000,16,-0.28675,2.47500,9690.27402,1001 days 08:00:00,-0.53177,-1.28584
0,BbandsCross,STOXX50m,16388,0.73140,-5.54591,9.81916,9445.40937,605,38.70968,0.00000,31,-0.08452,2.80000,9445.40937,1001 days 12:00:00,-0.56480,-1.77654
0,BbandsCross,JP225m,16386,0.86134,-12.10792,17.48232,8789.20818,779,40.00000,0.00000,75,-0.12301,2.66667,8789.20818,1030 days 10:00:00,-0.69258,-2.83710


In [ ]:
best_performers = wfo_performance[wfo_performance['custom_metric'] > 1]
best_performers

# Montecarlo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import montecarlo_statistics_simulation


# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []

dd_df = pd.DataFrame()
ret_df = pd.DataFrame()

for index, row in best_performers.iterrows():
    print(f"Procesando ticker: {row.ticker}")
    trades_history = stats_per_symbol[row.ticker][row.interval]._trades
    
    dd = pd.DataFrame()
    ret = pd.DataFrame()
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, dd, ret = montecarlo_statistics_simulation(trade_history=trades_history, n_simulations=100_000, initial_equity=100_000, threshold_ruin=0.8, return_raw_curves=True)
    
    display(mc)
    
    dd['ticker'] = row.ticker
    ret['ticker'] = row.ticker
    
    dd_df = pd.concat([dd_df, dd])
    ret_df = pd.concat([ret_df, ret])



In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 8))
sns.boxplot(data=dd_df, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = dd_df["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = dd_df["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(25, 8))
sns.boxplot(data=ret_df, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = ret_df["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = ret_df["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
